In [1]:
!wget -q https://dlcdn.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j
import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"# Start a SparkSession
import findspark
findspark.init()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=3417851756f0e85462ca83dbdd03dd28918fe50b9aa9c2554ade0dd561a93be3
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.show()



+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 2. Create a temporary view of the DataFrame.

df.createOrReplaceTempView('home_sales')


In [28]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""

    SELECT SUBSTRING(date, 1, 4) as year, ROUND(AVG(price), 2) as AverageHousePrice
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY 1
    SORT BY 1 ASC

""").show()


+----+-----------------+
|year|AverageHousePrice|
+----+-----------------+
|2019|         300263.7|
|2020|        298353.78|
|2021|        301819.44|
|2022|        296363.88|
+----+-----------------+



The average price of a four bedroom house sold in each year is:

\$300,263.70 in 2019 \\
\$298,353.78 in 2020 \\
\$301,819.44 in 2021 \\
\$296,363.88 in 2022 \\

In [26]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

spark.sql("""

      SELECT date_built, ROUND(AVG(price), 2) as AverageHousePrice
      FROM home_sales
      WHERE bedrooms = 3
      AND bathrooms = 3
      GROUP BY 1
      SORT BY 1 ASC



""").show()


+----------+-----------------+
|date_built|AverageHousePrice|
+----------+-----------------+
|      2010|        292859.62|
|      2011|        291117.47|
|      2012|        293683.19|
|      2013|        295962.27|
|      2014|        290852.27|
|      2015|         288770.3|
|      2016|        290555.07|
|      2017|        292676.79|
+----------+-----------------+



The average price of a three bedroom and three bathroom house built in each year is:

\$292,859.62 in 2010 \\
\$291,117.47 in 2011 \\
\$293,683.19 in 2012 \\
\$295,962.27 in 2013 \\
\$290,852.27 in 2014 \\
\$288,770.30 in 2015 \\
\$290,555.07 in 2016 \\
\$292,676.79 in 2017 \\


In [25]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

spark.sql("""

        SELECT date_built, ROUND(AVG(price), 2) as AverageHousePrice
        FROM home_sales
        WHERE bedrooms = 3
        AND bathrooms = 3
        AND floors = 2
        AND sqft_living >= 2000
        GROUP BY 1
        SORT BY 1 ASC


""").show()




+----------+-----------------+
|date_built|AverageHousePrice|
+----------+-----------------+
|      2010|        285010.22|
|      2011|        276553.81|
|      2012|        307539.97|
|      2013|        303676.79|
|      2014|        298264.72|
|      2015|        297609.97|
|      2016|         293965.1|
|      2017|        280317.58|
+----------+-----------------+



The average price of a three bedroom, three bathroom, and 2 floor house built in each year is:

\$\$ 285,010.22 in 2010 \
\$\$ 276,553.81 in 2011 \
\$\$ 307,539.97 in 2012 \
\$\$ 303,676.79 in 2013 \
\$\$ 298,264.72 in 2014 \
\$\$ 297,609.97 in 2015 \
\$\$ 293,965.10 in 2016 \
\$\$ 280,317.58 in 2017 


In [30]:
df.show(5)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [37]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""

        SELECT view, ROUND(AVG(price), 2) as AverageHousePrice
        FROM home_sales
        WHERE price >= 350000
        GROUP BY 1
        SORT BY 1 DESC


""").show()



print("--- %s seconds ---" % (time.time() - start_time))

+----+-----------------+
|view|AverageHousePrice|
+----+-----------------+
|  99|       1061201.42|
|  98|       1053739.33|
|  97|       1129040.15|
|  96|       1017815.92|
|  95|        1054325.6|
|  94|        1033536.2|
|  93|       1026006.06|
|  92|        970402.55|
|  91|       1137372.73|
|  90|       1062654.16|
|   9|        401393.34|
|  89|       1107839.15|
|  88|       1031719.35|
|  87|        1072285.2|
|  86|       1070444.25|
|  85|       1056336.74|
|  84|       1117233.13|
|  83|       1033965.93|
|  82|        1063498.0|
|  81|       1053472.79|
+----+-----------------+
only showing top 20 rows

--- 0.7046432495117188 seconds ---


In [38]:
# 7. Cache the the temporary table home_sales.

spark.sql("cache table home_sales")

DataFrame[]

In [39]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [51]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""

        SELECT view, ROUND(AVG(price), 2) as AverageHousePrice
        FROM home_sales
        WHERE price >= 350000
        GROUP BY 1
        SORT BY 1 DESC


""").show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+-----------------+
|view|AverageHousePrice|
+----+-----------------+
|  99|       1061201.42|
|  98|       1053739.33|
|  97|       1129040.15|
|  96|       1017815.92|
|  95|        1054325.6|
|  94|        1033536.2|
|  93|       1026006.06|
|  92|        970402.55|
|  91|       1137372.73|
|  90|       1062654.16|
|   9|        401393.34|
|  89|       1107839.15|
|  88|       1031719.35|
|  87|        1072285.2|
|  86|       1070444.25|
|  85|       1056336.74|
|  84|       1117233.13|
|  83|       1033965.93|
|  82|        1063498.0|
|  81|       1053472.79|
+----+-----------------+
only showing top 20 rows

--- 0.45383310317993164 seconds ---


Uncached runtime: 0.70 seconds \
Cached runtime: 0.45 seconds

In [53]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

df.write.partitionBy('date_built').mode('overwrite').parquet('home_sales_partitioned')



In [54]:
# 11. Read the formatted parquet data.

parti_df_parq = spark.read.parquet('home_sales_partitioned')


In [55]:
# 12. Create a temporary table for the parquet data.

parti_df_parq.createOrReplaceTempView('p_home_sales_p')


In [56]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""

        SELECT view, ROUND(AVG(price), 2) as AverageHousePrice
        FROM p_home_sales_p
        WHERE price >= 350000
        GROUP BY 1
        SORT BY 1 DESC


""").show()



print("--- %s seconds ---" % (time.time() - start_time))

+----+-----------------+
|view|AverageHousePrice|
+----+-----------------+
|  99|       1061201.42|
|  98|       1053739.33|
|  97|       1129040.15|
|  96|       1017815.92|
|  95|        1054325.6|
|  94|        1033536.2|
|  93|       1026006.06|
|  92|        970402.55|
|  91|       1137372.73|
|  90|       1062654.16|
|   9|        401393.34|
|  89|       1107839.15|
|  88|       1031719.35|
|  87|        1072285.2|
|  86|       1070444.25|
|  85|       1056336.74|
|  84|       1117233.13|
|  83|       1033965.93|
|  82|        1063498.0|
|  81|       1053472.79|
+----+-----------------+
only showing top 20 rows

--- 1.222876787185669 seconds ---


In [57]:
# 14. Uncache the home_sales temporary table.

spark.sql("uncache table home_sales")


DataFrame[]

In [58]:
# 15. Check if the home_sales is no longer cached

if spark.catalog.isCached("home_sales"):
  print("Home Sales table still cached")
else:
  print("Nothing Cached")

Nothing Cached
